<a href="https://colab.research.google.com/github/Anikrage/Visual_Product_Matcher/blob/main/TL_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymongo python-dotenv tensorflow Pillow requests tqdm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.9 MB/s eta 0:00:00


In [5]:
import os
import requests
import numpy as np
from PIL import Image
from io import BytesIO

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

In [7]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [8]:
from google.colab import userdata

Databse Init

In [9]:
username = userdata.get('MONGO_USERNAME')
password = userdata.get('MONGO_PASSWORD')
cluster = userdata.get('MONGO_CLUSTER')
database = userdata.get('MONGO_DATABAS')

In [10]:
db_url = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority"

In [11]:
client = MongoClient(db_url, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Connected")
except Exception as e:
    print(e)

Connected


In [12]:
db=client[database]
products_collection=db['products']

Model Init

In [13]:
model=ResNet50(
    weights='imagenet',include_top=False,
    pooling='avg',input_shape=(224,224,3)
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
print(f"Output shape: {model.output_shape}")

Output shape: (None, 2048)


In [15]:
def get_image_features(img_url):
  try:
    response = requests.get(img_url, timeout=10)
    img=Image.open(BytesIO(response.content))

    img = img.convert('RGB')
    img = img.resize((224,224))
    img_array=img_to_array(img)
    img_array=np.expand_dims(img_array,axis=0)
    img_array=preprocess_input(img_array)
    features=model.predict(img_array,verbose=0)
    features=features.flatten()

    return features
  except Exception as e:
    print(f"Errror: {e}")

Testing on 1 product

In [17]:
test=products_collection.find_one({'image_url':{'$exists':True}})
print(test['name'])

Basics Men Blue Striped T-shirt


In [19]:
test_features=get_image_features(test['image_url'])
print(test_features)

[0.06252243 0.2839572  0.23323105 ... 0.01845633 0.05787547 0.6622259 ]


Extracting Features of all products in DB

In [20]:
products=products_collection.find({'image_url':{'$exists':True}})
products=list(products)

In [24]:
from tqdm import tqdm

In [29]:
for i,p in tqdm(enumerate(products),total=len(products)):
  if 'feature_embed' in p:
    continue
  image_url=p['image_url']
  features=get_image_features(image_url)
  if features is not None:
    feature_list=features.tolist()
    products_collection.update_one(
        {'_id':p['_id']},
        {'$set':{'embedding':feature_list}}
    )

100%|██████████| 225/225 [01:28<00:00,  2.54it/s]


testing on irl data

In [57]:
test_url="https://imagescdn.peterengland.com/img/app/product/3/39831756-19499222.jpg"
test_features=get_image_features(test_url)

In [58]:
test_embed=test_features.reshape(1,-1)

In [59]:
db_embeds=products_collection.find({'embedding':{'$exists':True}})
db_embeds=list(db_embeds)

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
matches=[]
for p in db_embeds:
  p_embed=np.array(p['embedding']).reshape(1,-1)
  similar=cosine_similarity(test_embed,p_embed)

  matches.append({'name':p['name'],
                  'image_url':p['image_url'],
                  'similarity':similar})

In [62]:
matches.sort(key=lambda x:x['similarity'],reverse=True)

In [63]:
print(matches[0])

{'name': 'Mark Taylor Men Blue Striped Shirt', 'image_url': 'https://res.cloudinary.com/driwu0lv4/image/upload/v1759947787/product_matcher/products/9064.jpg', 'similarity': array([[0.60303607]])}


In [64]:
for i,m in enumerate(matches[:10],1):
  print(f"{i}. {m['name']} - {m['similarity']}")

1. Mark Taylor Men Blue Striped Shirt - [[0.60303607]]
2. Puma Men Blue & White T-shirt - [[0.55600898]]
3. Arrow Woman Solid Cream Dress - [[0.53978922]]
4. Genesis Black Casual Shirt - [[0.53536541]]
5. Peter England Men Dark Olive Green Trousers - [[0.52615644]]
6. Hanes Men Pack of 2 Half Contrast Piping Ringer T-shirts - [[0.51327921]]
7. Levis Men Comfort Style Grey Innerwear Vest - [[0.5080645]]
8. Chromozome Men Black Innerwear Vest - [[0.50603407]]
9. UCB Men's Super Star Print Purple T-shirt - [[0.50252917]]
10. Facit Men Bicep Tee Red Innerwear Vest - [[0.50232876]]
